<a href="https://colab.research.google.com/github/R3IYO/NLP/blob/main/M3_Part_I_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Group Members :     
###Reiyo
###Sayed Raheel Hussain
###Onkar Kunte

[link text](https://)##Assignment 3 Naïve Bayes and Sentiment Classification and Logistic Regression
Instructions
* Read the following Chapter 4: Naive Bayes and Sentiment Classification. Speech and Language Processing. Daniel Jurafsky & James H. Martin. Copyright © 2021. All rights reserved. Draft of September 21, 2021. I have tried to pull out relevant notes for you below, but it is encouraged that you read each chapter provided.
* Read the following Chapter 5: Logistic Regression. Speech and Language Processing. Daniel Jurafsky & James H. Martin. Copyright © 2021. All rights reserved. Draft of September 21, 2021. I have tried to pull out relevant notes for you below, but it is encouraged that you read each chapter provided.

Summary
Classification is one of the most important tasks of NLP and in machine learning. In NLP it often means the task of text categorization for both sentiment analysis, spam detection, and topic modeling. Naïve Bayes is often one of the first classification algorithms defined in NLP.  The intuition behind a classifier is lies at the underlying probability inferred by the Bayesian Inference, which uses Baye’s rule and conditional probabilities.

Here’s a reminder on Baye’s Rule:
P(y)=P(x)P(x)/(P(y))

We are saying “what is the probability of x given y”. Naïve Bayes is a generative model because there is an input that helps the model determine what the output could be. Said differently, “to train a generative model we first collect a large amount of data in some domain (e.g., think millions of images, sentences, or sounds, etc.) and then train a model to generate data like it.” [6]

So in the case of Naïve Bayes, we say given some word, what should be the class of the current word we are assessing? Contrastingly, discriminative models such as logistic regression, learn from features provided to the algorithm and then determine or predict what the class is. [7]


With Naïve Bayes, the assumption is that the probabilities are independent. We often call the Naïve Bayes classifier the bag-of-words approach. That’s because we are essentially throwing in the collection of words into a ‘bag’, selecting a word at random, and then calculating their frequency to use in the Bayesian Inference. Thus, context – the position of words -- is ignored and despite this, it turns out that the Naïve Bayes approach can be accurate and effective at determining whether an email is spam for example.

Back to bag of words. With bag of words, we assume that the position of the words are not relevant -- that dependency or context in the word phrase or sentence doesn’t matter. Relatedly, the naive Bayes assumption implies that the conditional probabilities are independent -- a rather strange assumption to make for words in a sentence! The equation for the naive Bayes classifier is outlined below:

You can use Naive Bayes by creating an index of words and walking through every word position in a test or corpus.


It can be useful to be able to classify new "test" documents using already classified "training" documents.  A common example is using a corpus of labeled spam and ham (non-spam) e-mails to predict whether or not a new document is spam.

For this Assignment, you can start with a spam/ham dataset, then predict the class of new documents (either withheld from the training dataset or from another source such as your own spam folder).   One example corpus:   https://spamassassin.apache.org/old/publiccorpus/

You may work alone or in a group on this project.  You're welcome to use any tools or approach that you like.  Due before our next meetup. Starter code provided below.

Test example is provided at the end.

Libraries you may wish to use

In [ ]:
import pandas as pd
import numpy as np
from os import makedirs, path, remove, rename, rmdir
from tarfile import open as open_tar
from shutil import rmtree
from urllib import request, parse
from glob import glob
from os import path
from re import sub
from email import message_from_file
from glob import glob
from sklearn.model_selection import StratifiedShuffleSplit
from collections import defaultdict
from functools import partial
from sklearn.metrics import (accuracy_score, f1_score, precision_score, recall_score)
from sklearn.model_selection import cross_val_predict
from sklearn.neighbors import KNeighborsClassifier
import gc
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_score, recall_score, confusion_matrix

download corpus using the following functions

Note: you may need to mount your drive on google then run this location. See previous exercises.

In [ ]:
def download_corpus(dataset_dir: str = 'data'):
    base_url = 'https://spamassassin.apache.org'
    corpus_path = 'old/publiccorpus'
    files = {
        '20021010_easy_ham.tar.bz2': 'ham',
        '20021010_hard_ham.tar.bz2': 'ham',
        '20021010_spam.tar.bz2': 'spam',
        '20030228_easy_ham.tar.bz2': 'ham',
        '20030228_easy_ham_2.tar.bz2': 'ham',
        '20030228_hard_ham.tar.bz2': 'ham',
        '20030228_spam.tar.bz2': 'spam',
        '20030228_spam_2.tar.bz2': 'spam',
        '20050311_spam_2.tar.bz2': 'spam' }

    #creates the folders: downloads, ham and spam
    downloads_dir = path.join(dataset_dir, 'downloads')
    ham_dir = path.join(dataset_dir, 'ham')
    spam_dir = path.join(dataset_dir, 'spam')

    makedirs(downloads_dir, exist_ok=True)
    makedirs(ham_dir, exist_ok=True)
    makedirs(spam_dir, exist_ok=True)


    for file, spam_or_ham in files.items():
        # download files from URL of each specific .bz2 file
        url = parse.urljoin(base_url, f'{corpus_path}/{file}')
        tar_filename = path.join(downloads_dir, file)
        request.urlretrieve(url, tar_filename)

        #list e-mails in the compressed .bz2 file
        emails = []
        with open_tar(tar_filename) as tar:
            tar.extractall(path=downloads_dir)
            for tarinfo in tar:
                if len(tarinfo.name.split('/')) > 1:
                    emails.append(tarinfo.name)

        # move e-mails to ham or spam directory
        for email in emails:
            directory, filename = email.split('/')
            directory = path.join(downloads_dir, directory)

            if not path.exists(path.join(dataset_dir, spam_or_ham, filename)):
                rename(path.join(directory, filename),
                   path.join(dataset_dir, spam_or_ham, filename))

        rmtree(directory)

download_corpus()

#How many e-mails are classified in our dataset as either Spam or not Spam?


In [ ]:
#How many e-mails are classified in our dataset as either Spam or not Spam?
ham_dir = path.join('data', 'ham')
spam_dir = path.join('data', 'spam')

print('Number of Non-Spam E-mails:', len(glob(f'{ham_dir}/*')))
print('\nNumber of Spam E-mails:', len(glob(f'{spam_dir}/*')))

Number of Non-Spam E-mails: 6952

Number of Spam E-mails: 2399


Provide your classifier below

In [ ]:
##### START CODE ####
import glob
import pandas as pd

ham_dir = path.join('data', 'ham')
spam_dir = path.join('data', 'spam')

# Create lists to store email content and labels
email_content = []
labels = []

# Process non-spam emails
print("Non-Spam E-mails:")
for file_path in glob.glob(f"{ham_dir}/*"):
    with open(file_path, 'rb') as file:  # Open the file in binary mode
        try:
            content = file.read().decode('utf-8')
        except UnicodeDecodeError:
            content = file.read().decode('latin-1')  # Try a different encoding if UTF-8 fails
        email_content.append(content)
        labels.append("ham")
        print(content)

# Process spam emails
print("\nSpam E-mails:")
for file_path in glob.glob(f"{spam_dir}/*"):
    with open(file_path, 'rb') as file:  # Open the file in binary mode
        try:
            content = file.read().decode('utf-8')
        except UnicodeDecodeError:
            content = file.read().decode('latin-1')  # Try a different encoding if UTF-8 fails
        email_content.append(content)
        labels.append("spam")
        print(content)

# Create a DataFrame from the email content and labels
df_sp_hm = pd.DataFrame({'Content': email_content, 'Label': labels})
print(df_sp_hm)

#### END CODE ####

Streaming output truncated to the last 5000 lines.
Dear All,

See www.digitalidentityforum.com for more details. Speakers include
Microsoft and Liberty Alliance, UK central and local government, law
enforcement, financial services (Egg and RBS/NatWest), EC Research Centre, a
psychologist and others. Look forward to seeing you there.

Regards,
Dave Birch.

-- 
-- David Birch, Director, Consult Hyperion
-- 
-- tel +44 (0)1483 301793, fax +44 (0)1483 561657
-- mail dave@chyp.com, web http://www.chyp.com
-- 
-- See you at the 2nd Annual Digital Transactions Forum in Singapore
-- October 16th/17th 2002, see http://www.digitaltransactionsforum.com/

--- end forwarded text


-- 
-----------------
R. A. Hettinga <mailto: rah@ibuc.com>
The Internet Bearer Underwriting Corporation <http://www.ibuc.com/>
44 Farquhar Street, Boston, MA 02131 USA
"... however it may deserve respect for its usefulness and antiquity,
[predicting the end of the world] has not been found agreeable to
experience." -- Ed

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



From fork-admin@xent.com  Mon Aug 26 19:34:22 2002
Return-Path: <fork-admin@xent.com>
Delivered-To: yyyy@localhost.netnoteinc.com
Received: from localhost (localhost [127.0.0.1])
	by phobos.labs.netnoteinc.com (Postfix) with ESMTP id 83D2443F9B
	for <jm@localhost>; Mon, 26 Aug 2002 14:34:22 -0400 (EDT)
Received: from phobos [127.0.0.1]
	by localhost with IMAP (fetchmail-5.9.0)
	for jm@localhost (single-drop); Mon, 26 Aug 2002 19:34:22 +0100 (IST)
Received: from xent.com ([64.161.22.236]) by dogma.slashnull.org
    (8.11.6/8.11.6) with ESMTP id g7QISdZ04518 for <jm@jmason.org>;
    Mon, 26 Aug 2002 19:28:40 +0100
Received: from lair.xent.com (localhost [127.0.0.1]) by xent.com (Postfix)
    with ESMTP id 6ECAE2940A2; Mon, 26 Aug 2002 11:26:09 -0700 (PDT)
Delivered-To: fork@spamassassin.taint.org
Received: from barrera.org (cpe-65-172-233-109.sanbrunocable.com
    [65.172.233.109]) by xent.com (Postfix) with ESMTP id C89C129409E for
    <fork@xent.com>; Mon, 26 Aug 2002 11:25:06 -0700 (P

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



The following email is a test email. You can take this and test your classifier to see if it predicts spam or not.



In [ ]:
df_sp_hm

,Content,Label
0,From fork-admin@xent.com Sun Sep 22 14:11:41 ...,ham
1,From rssfeeds@jmason.org Tue Sep 24 10:48:01 ...,ham
2,From razor-users-admin@lists.sourceforge.net ...,ham
3,,ham
4,From fork-admin@xent.com Fri Aug 23 11:08:39 ...,ham
...,...,...
9346,From paige_455@aol.com Sun Sep 22 14:13:09 20...,spam
9347,From fork-admin@xent.com Tue Aug 6 12:01:13 ...,spam
9348,From antheaygd@chinchilla.freeserve.co.uk Wed...,spam
9349,From sophia_komar@eudoramail.com Mon Jun 24 1...,spam


In [ ]:
df_sp_hm.groupby('Label').describe()

Content                
        count unique top freq
Label                        
ham      6952   6318      465
spam     2399   2106      288

In [ ]:
#converting spam/ham data into numerical form  by creating new column
df_sp_hm['spam']= df_sp_hm['Label'].apply(lambda x: 1 if x=='spam' else 0)

In [ ]:
df_sp_hm

,Content,Label,spam
0,From fork-admin@xent.com Sun Sep 22 14:11:41 ...,ham,0
1,From rssfeeds@jmason.org Tue Sep 24 10:48:01 ...,ham,0
2,From razor-users-admin@lists.sourceforge.net ...,ham,0
3,,ham,0
4,From fork-admin@xent.com Fri Aug 23 11:08:39 ...,ham,0
...,...,...,...
9346,From paige_455@aol.com Sun Sep 22 14:13:09 20...,spam,1
9347,From fork-admin@xent.com Tue Aug 6 12:01:13 ...,spam,1
9348,From antheaygd@chinchilla.freeserve.co.uk Wed...,spam,1
9349,From sophia_komar@eudoramail.com Mon Jun 24 1...,spam,1


In [ ]:
#create train/test split
x_train, x_test, y_train,y_test= train_test_split(df_sp_hm.Content, df_sp_hm.spam,test_size=0.25)

In [ ]:
x_train.describe()

count     7013
unique    6354
top           
freq       569
Name: Content, dtype: object

In [ ]:
#find the word count and store data as a matrix
cv =CountVectorizer()
x_train_count=cv.fit_transform(x_train.values)

In [ ]:
x_train_count.toarray()

array([[1, 0, 1, ..., 0, 0, 0],
       [6, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [3, 0, 1, ..., 0, 0, 0],
       [2, 0, 3, ..., 0, 0, 0],
       [3, 0, 1, ..., 0, 0, 0]])

In [ ]:
#train model
model =MultinomialNB()
model.fit(x_train_count,y_train)

MultinomialNB()

In [ ]:
# Transform test data using the same CountVectorizer object
x_test_count = cv.transform(x_test)

# Calculate accuracy score
accuracy = model.score(x_test_count, y_test)
print("Accuracy:", accuracy)

# Make predictions using the trained model
y_pred = model.predict(x_test_count)

# Compute precision score
precision = precision_score(y_test, y_pred)
print("Precision:", precision)

# Compute recall score
recall = recall_score(y_test, y_pred)
print("Recall:", recall)

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)


Accuracy: 0.9473909324208726
Precision: 0.9812889812889813
Recall: 0.8054607508532423
Confusion Matrix:
[[1743    9]
 [ 114  472]]


In [ ]:
spam_email = [
    """
    Subject: Get Rich Quick!

    Dear Friend,

    Congratulations! You've been selected to participate in an exclusive opportunity to make thousands of dollars from the comfort of your own home. Our revolutionary system guarantees quick and easy cash with minimal effort.

    No more struggling to pay bills or worrying about financial security. With our proven method, you can start earning massive amounts of money in no time.

    Here's what some of our satisfied customers have to say:
    - "I was skeptical at first, but I'm now living my dream life thanks to this incredible system!" - John S.
    - "I never thought making money online could be this simple. It's changed my life!" - Sarah L.

    Don't miss out on this limited-time offer. Act now to secure your spot and start enjoying a life of financial freedom.

    Click the link below to get started:
    www.getrichquick.com

    Remember, this opportunity is exclusive and won't last long. Take control of your financial future today!

    Best regards,
    The Get Rich Quick Team
    """
]

spam_email_count = cv.transform(spam_email)
prediction = model.predict(spam_email_count)

if (prediction[0]==1):
  print('spam mail')

else:
  print('ham mail')


spam mail
